목표

Aruomarker 생성 (기존 코드 활용)

Arucomarker 검출 (기존 코드 활용)

Arucomarker 카메라에서 인식

Arucomarker 상대 위치 인식

Arucomarker 정밀도 수정 연습

In [5]:
import numpy as np
import cv2 as cv
import pickle

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((5*8,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:5].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

capture = cv.VideoCapture(0)
while True:
    ret, frame = capture.read()
    # cv.imshow("VideoFrame", frame)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,5), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(frame, (8,5), corners2, ret)
        cv.imshow('VideoFrame', frame)
    else:
        cv.imshow("VideoFrame", frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

with open('objpoints.pk', 'wb') as f:
    pickle.dump(objpoints, f)
with open('imgpoints.pk', 'wb') as f:
    pickle.dump(imgpoints, f)
with open('gray.pk', 'wb') as f:
    pickle.dump(gray, f)

cv.destroyAllWindows()

In [1]:
import numpy as np
import imutils
import cv2 as cv
import sys

In [2]:
# OpenCV에서 지원하는 ARUCO MARKER의 TYPE에 대한 딕셔너리
ARUCO_DICT = {
	"DICT_4X4_50": cv.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv.aruco.DICT_APRILTAG_36h11
}

In [3]:
import pickle

with open('objpoints.pk', 'rb') as f:
    objpoints = pickle.load(f)
with open('imgpoints.pk', 'rb') as f:
    imgpoints = pickle.load(f)
with open('gray.pk', 'rb') as f:
    gray = pickle.load(f)

In [4]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera (objpoints, imgpoints, gray.shape[::-1], None , None )

In [81]:
def pose_esitmation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):

    '''
    frame - 카메라 또는 비디오에서 읽어온 이미지
    aruco_dict_type - 사용하는 MARKER의 TYPE (예. 앞서 설명한 코드의 ARUCO_DICT[aruco_type])
    matrix_coefficients - 카메라 Calibration 과정 후에 획득한 Intrinsic matrix (내부 행렬)
    distortion_coefficients - 카메라 Calibration 과정 후에 획득한 Distortion coefficients (왜곡 계수)
    return:
    frame - 읽어온 이미지 + MARKER의 축
    '''

    # 카메라 또는 비디오에서 읽어온 이미지를 흑백으로 변환
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # 정의한 ARUCO MARKER의 TYPE을
    # 컴퓨터가 이해할 수 있는 형태의 언어로 변환하는 부분임
    cv.aruco_dict = cv.aruco.Dictionary_get(aruco_dict_type)
    # 검출을 위한 파라미터 설정하는 부분임
    # 일반적으로 default 값을 그대로 사용함
    parameters = cv.aruco.DetectorParameters_create()
    # MARKER 검출
    corners, ids, rejected_img_points = cv.aruco.detectMarkers(gray, cv.aruco_dict,parameters=parameters)
        # cameraMatrix=matrix_coefficients,
        # distCoeff=distortion_coefficients)

    # 하나 이상의 MARKER가 검출됐을 때만 실행되는 부분
    if len(corners) > 0:
        font=cv.FONT_HERSHEY_SIMPLEX
        for i in range(0, len(ids)):
            # 각 MARKER의 Pose를 측정 하고 rvec 과 tvec 으로 반환
            rvec, tvec, markerPoints = cv.aruco.estimatePoseSingleMarkers(corners[i], 0.1, matrix_coefficients,
                                                                       distortion_coefficients)
            # MARKER 테두리 그려주기 (삭제해도 됨)
            cv.aruco.drawDetectedMarkers(frame, corners) 

            # MARKER 축 그려주기 (삭제해도 됨)
            cv.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)

            rotMat = np.zeros((3, 3), np.float32)
            rotMat,_ = cv.Rodrigues(rvec)

            for i in range(3):
                rot = '%2.3f, %2.3f, %2.3f' %(rotMat[i][0], rotMat[i][1], rotMat[i][2])
                cv.putText(frame,rot,(10, 60 + 30 * i),font,1,(0,0,255),1)

            pos = '%2.3f, %2.3f, %2.3f'%(tvec[0,0,0], tvec[0,0,1], tvec[0,0,2])
            cv.putText(frame, pos,(10, 30),font,1,(255,0,0),1)
    else:
        rotMat = []
    return frame, rotMat

In [6]:
# 인식할 MARKER의 TYPE을 정의해줌
aruco_type = "DICT_4X4_50"
print("[INFO] detecting '{}' tags...".format(aruco_type))
arucoDict = cv.aruco.Dictionary_get(ARUCO_DICT[aruco_type])

[INFO] detecting 'DICT_4X4_50' tags...


In [83]:
import cv2 as cv

capture = cv.VideoCapture(1)
# capture.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
# capture.set(cv.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = capture.read()
    # cv.imshow("VideoFrame", frame)
    
    det_frame, rotMat = pose_esitmation(frame, ARUCO_DICT[aruco_type], mtx, dist)

    cv.imshow("VideoFrame", det_frame)

    if rotMat != []:
        r = rotMat.flatten()
        print(r[0])

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cv.destroyAllWindows()
capture.release()

<ipython-input-83-794c277c4538>:15: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if rotMat != []:


0.9898296847223941
0.988238608153216
0.986876442236291
0.9869953702831485
0.9907101431886866
0.9920560520573282
0.9927002515128414
0.9953860394410758
0.9956295425186455
0.9956706020543525
0.9962689671785768
0.9952379197923126
0.9947099487536981
0.9915347091437675
0.9917311231743414
0.9921688571688884
0.9926719777832527
0.9918461732034506
0.9891845176574929
0.9876307835221039
0.9964174005585352
0.9980690572840981
0.9979312844425575
0.9988133766209384
0.9990194242638581
0.9988198683425971
0.9986009944185246
0.9983039226323476
0.9986022047693223
0.9982253659646088
0.9975081201926709
0.9968460117685919
0.9971414473269747
0.9976930999001539
0.9983935272705404
0.9983812252229088
0.9986647381159135
0.9984913225061862
0.9982461193114868
0.9982954228583475
0.9988356633310651
0.9988882193590776
0.9995235285757376
0.9991817482808113
0.9992263114293628
0.9983472377296855
0.9975934561582055
0.9970594007797164
0.9976042601162879
0.9982762603444085
0.9986591272876935
0.997866069476997
0.9974910478139